In [1]:
import pandas as pd


train_data = pd.read_csv("data\main\df_train_b1.csv", index_col=0)
test_data = pd.read_csv('data\main\df_test_b1.csv', index_col=0)

In [2]:
if True:

    import pandas as pd
    temp_df = pd.read_csv("./data/detail/df_loan1_b1.csv")

    def chuli_loan(df, colname, ways):
        cust_id = list(set(df['cust_id']))
        res_df = pd.DataFrame({'cust_id': cust_id})
        try:
            temp_train = pd.read_csv('./data/df_train.csv')
            temp_test = pd.read_csv('./data/df_test.csv')
            temp = list(temp_test.columns)
        except:
            temp = []

        ways_1 = []

        global train_data, test_data

        for i in ways:
            if (colname+'_'+i) not in temp:
                ways_1.append(i)
                res_df[colname+'_'+i] = None
            else:
                print(colname+'_'+i+'已存在')
                train_data = pd.merge(train_data, temp_train[['cust_id', colname+'_'+i]], on="cust_id",
                                      how="left", suffixes=("", "_y"))
                test_data = pd.merge(test_data, temp_test[['cust_id', colname+'_'+i]], on="cust_id",
                                     how="left", suffixes=("", "_y"))
        ways = ways_1
        if len(ways) == 0:
            return res_df
        if 'mean' in ways:
            res_df[colname+'_mean'] = df.groupby('cust_id')[colname].mean()
        if 'sum' in ways:
            res_df[colname+'_sum'] = df.groupby('cust_id')[colname].sum()
        if 'count' in ways:
            res_df[colname+'_count'] = df.groupby('cust_id')[colname].count()
        if 'std' in ways:
            res_df[colname+'_std'] = df.groupby('cust_id')[colname].std()
        if 'diff_std' in ways:
            res_df[colname+'_diff_std'] = df.groupby('cust_id')[colname].apply(
                lambda x: x.sort_values(ascending=True).diff()[2:].std())
        return res_df

    for colname in ['loan1_2', 'loan1_3', 'loan1_7', 'loan1_9']:
        df = chuli_loan(temp_df, colname, ('mean', 'sum'))
        train_data = pd.merge(train_data, df, on="cust_id",
                              how="left", suffixes=("", "_y"))
        test_data = pd.merge(test_data, df, on="cust_id",   
                             how="left", suffixes=("", "_y"))
    for colname in ['loan1_2','loan1_3']:
        df = chuli_loan(temp_df,colname , ('count',))
        train_data = pd.merge(train_data, df, on="cust_id",
                                how="left", suffixes=("", "_y"))
        test_data = pd.merge(test_data, df, on="cust_id",
                                how="left", suffixes=("", "_y"))                    

loan1_2_sum已存在
loan1_3_sum已存在
loan1_7_sum已存在
loan1_9_sum已存在


In [3]:
df_train=train_data
df_test=test_data

In [4]:
df_train.drop(["loan1_17",'loan1_31'],axis=1,inplace=True)
df_test.drop(["loan1_17",'loan1_31'],axis=1,inplace=True)

In [5]:
def delnan(df):
    df = df.dropna(how="all", axis=1)  # 删除全是空值的列
    df = df.fillna(-99)
    return df

df_train = delnan(train_data)
df_test = delnan(test_data)

In [6]:
full_col_name = list(df_train.columns)
basic_col = [i for i in full_col_name if "basic" in i]
loan_col = [i for i in full_col_name if "loan" in i]
overdue_col = [i for i in full_col_name if "overdue" in i]
query_col = [i for i in full_col_name if "query" in i]
judicial_col = [i for i in full_col_name if "judicial" in i]
date_col = [i for i in full_col_name if "date" in i]

In [7]:
x_col = basic_col+['is_judicial']+loan_col+query_col+overdue_col+date_col
x_col=[i for i in x_col if i in full_col_name]

In [8]:
print(x_col)

['basic_1', 'basic_2', 'basic_3', 'basic_4', 'basic_5', 'basic_6', 'basic_7', 'basic_8', 'basic_9', 'basic_10', 'basic_11', 'basic_12', 'basic_13', 'basic_14', 'basic_15', 'loan1_1', 'loan1_2', 'loan1_3', 'loan1_4', 'loan1_5', 'loan1_6', 'loan1_7', 'loan1_8', 'loan1_9', 'loan1_10', 'loan1_11', 'loan1_12', 'loan1_13', 'loan1_14', 'loan1_15', 'loan1_16', 'loan1_18', 'loan1_19', 'loan1_20', 'loan1_21', 'loan1_22', 'loan1_23', 'loan1_24', 'loan1_25', 'loan1_26', 'loan1_27', 'loan1_28', 'loan1_29', 'loan1_30', 'loan1_32', 'loan1_33', 'loan2_1', 'loan2_2', 'loan2_3', 'loan2_4', 'loan2_5', 'loan2_6', 'loan2_7', 'loan2_8', 'loan2_9', 'loan2_10', 'loan2_11', 'loan2_12', 'loan1_2_sum', 'loan1_2_mean', 'loan1_3_sum', 'loan1_3_mean', 'loan1_7_sum', 'loan1_7_mean', 'loan1_9_sum', 'loan1_9_mean', 'loan1_2_count', 'loan1_3_count', 'query_1', 'query_2', 'query_3', 'query_4', 'query_5', 'query_6', 'query_7', 'query_8', 'query_9', 'query_10', 'query_11', 'overdue_1', 'overdue_2', 'overdue_3', 'overdue_4

In [9]:
import copy
import xgboost as xgb
import random
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation


In [10]:
X = df_train[x_col]
Y = df_train['label']

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, Y, test_size=3000, stratify=Y, random_state=2022)

In [16]:

dtrain=xgb.DMatrix(X,label=Y)

param = {'max_depth': 5,
         'eta': 0.008,
         'objective': 'binary:logistic',
         "eval_metric":"auc",
         "scale_pos_weight":1,
         "subsample":0.8,
         "min_child_weight":1,
         'tree_method': 'gpu_hist',
         'gpu_id': 0,}
         

cv_res= xgb.cv(param,dtrain,num_boost_round=2000,early_stopping_rounds=200,nfold=10,show_stdv=True)
print(cv_res)

      train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0           0.604900       0.017927       0.562451      0.029307
1           0.630291       0.018394       0.591459      0.033039
2           0.665921       0.016357       0.618324      0.033117
3           0.681037       0.018176       0.626597      0.031816
4           0.690441       0.015471       0.629628      0.032696
...              ...            ...            ...           ...
1290        0.983406       0.001329       0.741055      0.018793
1291        0.983443       0.001328       0.741025      0.018667
1292        0.983469       0.001323       0.741052      0.018717
1293        0.983500       0.001327       0.741070      0.018713
1294        0.983523       0.001324       0.741179      0.018687

[1295 rows x 4 columns]


In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
x_perd = xgb.DMatrix(df_test[x_col])


In [11]:


dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
x_perd = xgb.DMatrix(df_test[x_col])

evallist = [(dtrain,"train"),(dtest, 'eval')]
param = {
    'max_depth': 5,
    'eta': 0.008,
    'colsample_bytree': 0.5,
    'objective': 'binary:logistic',
    # 'verbosity':0,
    'scale_pos_weight': 1,
    'eval_metric': 'auc',
    'subsample': 0.55,
    'min_child_weight': 10,
    'gamma': 0.5,
    'seed': 666,
    'tree_method': 'gpu_hist',
    'gpu_id': 0,
}

bst = xgb.train(param, dtrain, 3000, evals=evallist,early_stopping_rounds=300)

[0]	train-auc:0.58742	eval-auc:0.52430
[1]	train-auc:0.58742	eval-auc:0.52430
[2]	train-auc:0.58788	eval-auc:0.53132
[3]	train-auc:0.62922	eval-auc:0.55017
[4]	train-auc:0.64781	eval-auc:0.55371
[5]	train-auc:0.64880	eval-auc:0.54972
[6]	train-auc:0.64860	eval-auc:0.55671
[7]	train-auc:0.64860	eval-auc:0.55671
[8]	train-auc:0.65111	eval-auc:0.56149
[9]	train-auc:0.65166	eval-auc:0.55987
[10]	train-auc:0.65176	eval-auc:0.55640
[11]	train-auc:0.65474	eval-auc:0.56364
[12]	train-auc:0.65494	eval-auc:0.56225
[13]	train-auc:0.66141	eval-auc:0.56374
[14]	train-auc:0.66580	eval-auc:0.56534
[15]	train-auc:0.67191	eval-auc:0.57713
[16]	train-auc:0.67406	eval-auc:0.58022
[17]	train-auc:0.67302	eval-auc:0.57441
[18]	train-auc:0.67553	eval-auc:0.57671
[19]	train-auc:0.67604	eval-auc:0.57824
[20]	train-auc:0.68172	eval-auc:0.58533
[21]	train-auc:0.68397	eval-auc:0.58748
[22]	train-auc:0.68238	eval-auc:0.58512
[23]	train-auc:0.68408	eval-auc:0.58548
[24]	train-auc:0.68357	eval-auc:0.58572
[25]	train

In [12]:
bst.predict(x_perd)

array([0.02047139, 0.00405599, 0.01634305, ..., 0.07123405, 0.163696  ,
       0.01588306], dtype=float32)

In [13]:
xgb_pred_label = list(bst.predict(x_perd))

In [14]:
lgbcallback = [early_stopping(300), log_evaluation(10)]
param = {
        'task': 'train',
        "max_depth": 5,
        'boosting_type': 'gbdt',  # 设置提升类型
        'objective': 'binary',  # 目标函数
        'metric': 'auc',  # 评估函数
        'learning_rate': 0.01,  # 学习速率
        # 建树的特征选择比例
        'feature_fraction': 1,
        'bagging_fraction': 0.7,  # 建树的样本采样比例
        'bagging_freq': 1,  # k 意味着每 k 次迭代执行bagging
        'min_data_in_leaf': 1,
        'verbose': -1,  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
        'device_type': 'gpu',
        'seed':666,
    }

dtrain = lgb.Dataset(X_train, y_train)
dtest = lgb.Dataset(X_test, y_test)
bst1 = lgb.train(param, dtrain, 3000, valid_sets=[
            dtest], callbacks=lgbcallback)

Training until validation scores don't improve for 300 rounds
[10]	valid_0's auc: 0.626204
[20]	valid_0's auc: 0.629657
[30]	valid_0's auc: 0.641903
[40]	valid_0's auc: 0.645592
[50]	valid_0's auc: 0.647683
[60]	valid_0's auc: 0.657132
[70]	valid_0's auc: 0.65771
[80]	valid_0's auc: 0.661848
[90]	valid_0's auc: 0.67234
[100]	valid_0's auc: 0.671971
[110]	valid_0's auc: 0.675009
[120]	valid_0's auc: 0.678324
[130]	valid_0's auc: 0.677272
[140]	valid_0's auc: 0.679766
[150]	valid_0's auc: 0.682722
[160]	valid_0's auc: 0.684698
[170]	valid_0's auc: 0.68554
[180]	valid_0's auc: 0.688886
[190]	valid_0's auc: 0.69093
[200]	valid_0's auc: 0.690774
[210]	valid_0's auc: 0.691819
[220]	valid_0's auc: 0.694089
[230]	valid_0's auc: 0.695195
[240]	valid_0's auc: 0.697259
[250]	valid_0's auc: 0.697242
[260]	valid_0's auc: 0.698389
[270]	valid_0's auc: 0.700967
[280]	valid_0's auc: 0.70274
[290]	valid_0's auc: 0.703629
[300]	valid_0's auc: 0.703775
[310]	valid_0's auc: 0.705081
[320]	valid_0's auc: 0

In [21]:
bst1.predict(df_test[x_col])

array([0.01480733, 0.00437024, 0.01826027, ..., 0.03282345, 0.11009986,
       0.00590034])

In [15]:
lgb_pred_label=list(bst1.predict(df_test[x_col]))

In [35]:
import numpy as np

from catboost import CatBoostClassifier, Pool

# initialize data


model = CatBoostClassifier(iterations=3000,
                           depth=3,
                           learning_rate=0.04,
                           loss_function='Logloss',
                           subsample=0.8,
                           early_stopping_rounds=300,
                           verbose=True)
# train the model
model.fit(X_train, y_train,eval_set=(X_test,y_test))
# make the prediction using the resulting model

cat_pred_label = model.predict_proba(df_test[x_col])




0:	learn: 0.6327258	test: 0.6328302	best: 0.6328302 (0)	total: 6.02ms	remaining: 18.1s
1:	learn: 0.5790737	test: 0.5792295	best: 0.5792295 (1)	total: 11.6ms	remaining: 17.4s
2:	learn: 0.5311480	test: 0.5313250	best: 0.5313250 (2)	total: 17.5ms	remaining: 17.5s
3:	learn: 0.4888607	test: 0.4891061	best: 0.4891061 (3)	total: 24.2ms	remaining: 18.1s
4:	learn: 0.4513035	test: 0.4516160	best: 0.4516160 (4)	total: 31ms	remaining: 18.5s
5:	learn: 0.4181168	test: 0.4183946	best: 0.4183946 (5)	total: 37.6ms	remaining: 18.8s
6:	learn: 0.3885013	test: 0.3887152	best: 0.3887152 (6)	total: 45.2ms	remaining: 19.3s
7:	learn: 0.3624180	test: 0.3627203	best: 0.3627203 (7)	total: 51.2ms	remaining: 19.2s
8:	learn: 0.3393277	test: 0.3396825	best: 0.3396825 (8)	total: 58ms	remaining: 19.3s
9:	learn: 0.3186508	test: 0.3189765	best: 0.3189765 (9)	total: 64.4ms	remaining: 19.2s
10:	learn: 0.3005839	test: 0.3009179	best: 0.3009179 (10)	total: 69.8ms	remaining: 19s
11:	learn: 0.2842017	test: 0.2845988	best: 0.28

In [36]:
roc_auc_score(y_test,model.predict_proba(X_test)[:,1])

0.7202397867359503

In [18]:
pred_labels=[lgb_pred_label,xgb_pred_label]

In [19]:
final_pred_label = np.exp(
    np.mean(np.log(pred_labels), axis=0))

In [20]:
import time

date = time.strftime("%Y%m%d-%H%M%S", time.localtime())

res = pd.DataFrame({
    'cust_id': list(df_test["cust_id"]),
    'label': lgb_pred_label,
})
res.to_csv("./output/result"+date+".csv", encoding="utf-8", index=False)